In [6]:
from time_lstm import T1TimeLSTM, T2TimeLSTM
from dataset_delay_predictions import Dataset_Delay_Prediction, Dataset_Delay_Prediction_from_list, DatasetDelayPredictionStackOverflow
from time_dependant_representation import TimeDepMasking, TimeDepJointEmbedding
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Masking, Layer, LSTM
import os

In [7]:
import tensorflow as tf

In [10]:
import pickle

In [11]:
dataset_names = [
    '2019-10_11_12-dataset_delay_prediction',
    'stack-overflow-dataset'
]

In [ ]:
dataset_name = dataset_names[0]
pickle_filename = dataset_name.replace('/', '-')
if os.path.exists(pickle_filename):
    print('Reading pickle file...')
    dataset = pickle.load(open(pickle_filename, 'rb'))

Reading pickle file...


In [22]:
X_train = dataset.full_features_dt[dataset.training_set_length:]
seqlen = dataset.full_seqlen[dataset.training_set_length:]
y_train = np.array(dataset.full_values[dataset.training_set_length:])
print(y_train.shape)

(5307, 1)


In [23]:
number_of_event = dataset.number_of_event
X_train_bis = []
number_of_event = len(X_train[0][0])
for idx in range(len(X_train)):
    seq = X_train[idx]
    ts_list = [[a[-1]] for a in seq]
    X_train_bis.append(ts_list)

padding_value = 0.123456789
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_train_bis,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')

In [24]:
print(padded_inputs.shape)
lstm_units = 50

(5307, 735, 1)


In [25]:
regressor = Sequential()
regressor.add(Masking(mask_value=padding_value))
regressor.add(LSTM(units=lstm_units))
regressor.add(Dense(units=y_train.shape[1], activation='sigmoid'))

In [26]:
number_of_epochs = 5
regressor.compile(optimizer='adam', loss='mean_squared_error')
history = regressor.fit(padded_inputs, y_train, batch_size=50, epochs=number_of_epochs, verbose=2)

Train on 5307 samples
Epoch 1/5
5307/5307 - 37s - loss: 0.2308
Epoch 2/5
5307/5307 - 29s - loss: 0.2299
Epoch 3/5
5307/5307 - 28s - loss: 0.2292
Epoch 4/5
5307/5307 - 28s - loss: 0.2291
Epoch 5/5
5307/5307 - 28s - loss: 0.2290


In [30]:
X_test = dataset.full_features_dt[:dataset.training_set_length]
seqlen_test = dataset.full_seqlen[:dataset.training_set_length]
y_test = np.array(dataset.full_values[:dataset.training_set_length])
X_test_bis = []
number_of_event = len(X_train[0][0])
for idx in range(dataset.training_set_length):
    seq = X_test[idx]
    ts_list = [[a[-1]] for a in seq]
    X_test_bis.append(ts_list)


In [31]:
padding_value = 0.123456789
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_test_bis,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')

In [32]:
pred = regressor.predict(padded_inputs)

In [38]:
mean_sum_of_squares = sum([(y_test[i] - pred[i])**2 for i in range(dataset.training_set_length)])/dataset.training_set_length

In [39]:
print(mean_sum_of_squares)

[0.22825082]
